파이토치(PyTorch)에서는 텍스트에 대한 여러 추상화 기능을 제공하는 자연어 처리 라이브러리 토치텍스트(Torchtext)를 제공
---
* 파일 로드하기(File Loading) : 다양한 포맷의 코퍼스를 로드합니다.
* 토큰화(Tokenization) : 문장을 단어 단위로 분리해줍니다.
* 단어 집합(Vocab) : 단어 집합을 만듭니다.
* 정수 인코딩(Integer encoding) : 전체 코퍼스의 단어들을 각각의 고유한 정수로 맵핑합니다.
* 단어 벡터(Word Vector) : 단어 집합의 단어들에 고유한 임베딩 벡터를 만들어줍니다. 랜덤값으로 초기화한 값일 수도 있고, 사전 훈련된 임베딩 벡터들을 로드할 수도 있습니다.
* 배치화(Batching) : 훈련 샘플들의 배치를 만들어줍니다. 이 과정에서 패딩 작업(Padding)도 이루어집니다.

In [7]:
# !pip install torchtext

In [8]:
from torchtext.legacy.data import TabularDataset

## 1. 훈련 데이터와 테스트 데이터로 분리하기

In [9]:
# 영화 사이트 IMDB의 리뷰 데이터
import urllib.request
import pandas as pd
# urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

In [10]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


1은 긍정, 0은 부정

In [11]:
print('전체 샘플의 개수 : {}'.format(len(df)))

전체 샘플의 개수 : 50000


In [12]:
train_df = df[:25000]
test_df = df[25000:]
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

## 2. 필드 정의하기(torchtext.data)

* sequential : 시퀀스 데이터 여부. (True가 기본값)
* use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
* tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
* lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
* batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부.(False가 기본값)
* is_target : 레이블 데이터 여부. (False가 기본값)
* fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.

In [16]:
from torchtext.legacy import data # torchtext.data 임포트

# 필드 정의

# 실제 텍스트를 위한 text 객체
TEXT = data.Field(sequential=True, #시퀀스 데이터
                  use_vocab=True, # 단어 집합 만든다
                  tokenize=str.split, # str으로 자르는 토큰화
                  lower=True, # 전부 소문자
                  batch_first=True, # 미니 배치 차원을 맨 앞으로해서 데이터 부름
                  fix_length=20) # 최대 허용 길이, 이 길이에 맞춰 패딩작업 진행

# 레이블 데이터를 위한 label 객체
LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

## 3. 데이터셋 만들기
TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행

In [17]:
from torchtext.legacy.data import TabularDataset

train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

# fields : 위에서 정의한 필드를 지정. 첫번째 원소는 데이터 셋 내에서 해당 필드를 호칭할 이름, 두번째 원소는 지정할 필드.
# skip_header : 데이터의 첫번째 줄은 무시.

print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

훈련 샘플의 개수 : 25000
테스트 샘플의 개수 : 25000


In [23]:
print(vars(train_data[0])) # 주어진 인덱스 샘플 확인

{'text': ['my', 'family', 'and', 'i', 'normally', 'do', 'not', 'watch', 'local', 'movies', 'for', 'the', 'simple', 'reason', 'that', 'they', 'are', 'poorly', 'made,', 'they', 'lack', 'the', 'depth,', 'and', 'just', 'not', 'worth', 'our', 'time.<br', '/><br', '/>the', 'trailer', 'of', '"nasaan', 'ka', 'man"', 'caught', 'my', 'attention,', 'my', 'daughter', 'in', "law's", 'and', "daughter's", 'so', 'we', 'took', 'time', 'out', 'to', 'watch', 'it', 'this', 'afternoon.', 'the', 'movie', 'exceeded', 'our', 'expectations.', 'the', 'cinematography', 'was', 'very', 'good,', 'the', 'story', 'beautiful', 'and', 'the', 'acting', 'awesome.', 'jericho', 'rosales', 'was', 'really', 'very', 'good,', "so's", 'claudine', 'barretto.', 'the', 'fact', 'that', 'i', 'despised', 'diether', 'ocampo', 'proves', 'he', 'was', 'effective', 'at', 'his', 'role.', 'i', 'have', 'never', 'been', 'this', 'touched,', 'moved', 'and', 'affected', 'by', 'a', 'local', 'movie', 'before.', 'imagine', 'a', 'cynic', 'like', 'me

In [24]:
print(train_data[0]) 

In [25]:
print(train_data.fields.items())

dict_items([('text', <torchtext.legacy.data.field.Field object at 0x00000267AEB8C310>), ('label', <torchtext.legacy.data.field.Field object at 0x00000267AEB8C280>)])


## 4. 단어 집합(Vocabulary) 만들기

토큰화 전처리를 끝냈다면, 이제 각 단어에 고유한 정수를 맵핑해주는 정수 인코딩(Integer enoding) 작업이 필요합니다. 그리고 이 전처리를 위해서는 우선 단어 집합을 만들어주어야 합니다.

In [34]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)
# min_freq : 단어 집합에 추가 시 단어의 최소 등장 빈도 조건을 추가.
# max_size : 단어 집합의 최대 크기를 지정

print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

단어 집합의 크기 : 10002


In [39]:
# print(TEXT.vocab.stoi)
print(type(TEXT.vocab.stoi))
print(dict(list(TEXT.vocab.stoi.items())[:100]))
# max size = 10002개인 이유 : unk, pad 들어갔기 때문!

<class 'collections.defaultdict'>
{'<unk>': 0, '<pad>': 1, 'the': 2, 'a': 3, 'and': 4, 'of': 5, 'to': 6, 'is': 7, 'in': 8, 'i': 9, 'this': 10, 'it': 11, 'that': 12, '/><br': 13, 'was': 14, 'as': 15, 'for': 16, 'with': 17, 'but': 18, 'on': 19, 'movie': 20, 'his': 21, 'not': 22, 'are': 23, 'you': 24, 'film': 25, 'have': 26, 'he': 27, 'be': 28, 'at': 29, 'one': 30, 'by': 31, 'an': 32, 'they': 33, 'from': 34, 'who': 35, 'all': 36, 'like': 37, 'so': 38, 'just': 39, 'or': 40, 'has': 41, 'about': 42, "it's": 43, 'if': 44, 'her': 45, 'some': 46, 'out': 47, 'what': 48, 'very': 49, 'when': 50, 'more': 51, 'there': 52, 'even': 53, 'would': 54, 'my': 55, 'good': 56, 'she': 57, 'their': 58, 'only': 59, 'no': 60, 'really': 61, 'can': 62, 'up': 63, 'had': 64, 'which': 65, 'see': 66, 'were': 67, 'than': 68, '-': 69, 'we': 70, 'been': 71, 'into': 72, 'get': 73, 'much': 74, 'will': 75, 'because': 76, 'story': 77, 'most': 78, 'how': 79, 'other': 80, 'its': 81, 'first': 82, "don't": 83, 'time': 84, 'also'

## 5. 토치텍스트의 데이터로더 만들기

In [40]:
from torchtext.legacy.data import Iterator

batch_size = 5

train_loader = Iterator(dataset=train_data, batch_size = batch_size)
test_loader = Iterator(dataset=test_data, batch_size = batch_size)
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 5000
테스트 데이터의 미니 배치 수 : 5000


In [41]:
batch = next(iter(train_loader)) # 첫번째 미니배치
print(type(batch))


<class 'torchtext.legacy.data.batch.Batch'>


일반적인 데이터로더는 미니 배치를 텐서로 가져오지만 토치텍스트의 데이터로더는 torchtext.data.batch.Batch 객체를 가져온다.


In [42]:
print(batch.text)

tensor([[  10,    7,   30,    5,  134,  127,  431,   16,  986, 2259,   39,    0,
           17,  482,  287,   42,   11,    7, 1701, 2610],
        [  44,   24,   67,  465,   31,  148,    0,    0,   69,   97,  407,   37,
           10,   20,    0,    0,    2, 9855,    0,    7],
        [1337,   98,    2,   82, 6094,   19,    2, 2926,   54,   28,  962,  221,
          192, 8842,    6,   90,   63,  680,   42,  112],
        [  11,   61,    0,   88,   12,   11,  178, 1055,    2,    0,  205,   28,
           61, 4331,    8, 3870,    9,   62,   98,    5],
        [2035, 7605,    7,    2,  499,  814, 1404,    0,    0,  110,    0,    0,
            0, 1152,  872, 4504,   21,    0,   76,   60]])


## 6. <pad>토큰이 사용되는 경우

맨 처음 필드를 정의할 때 fix_length를 20이 아니라 150으로 정의하면
아래와 같은 결과로 나온다.


In [ ]:
'''
tensor([ 248,   39,    0,    0,   55, 7701,    0,  174,  701,   34,    3,  403,
           8,    0, 1480,    0, 2595, 1499,    0,    9,  388, 5068,    6,   73,
           ... 중략 ...
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1])
'''